In [26]:
import pandas as pd

In [27]:
name = 'VOLTAS'

In [28]:
def files(name: str, no: int):
    df = pd.read_parquet(f'order_{no}.parquet')
    df = df[df['Symbol'] == name]
    return len(df) > 0

def files_seq(name: str):
    i = 1
    res = []
    while True:
        try:
            if files(name, i):
                res.append(i)
            i += 1
        except FileNotFoundError:
            break

        if i > 5:
            break
    return res


print(files_seq(name))





[4, 5]


In [29]:
df1 = pd.read_parquet('order_4.parquet')
df2 = pd.read_parquet('order_5.parquet')

In [30]:
df = pd.concat([df1, df2])


In [31]:
print(len(df) == len(df1) + len(df2))

True


In [32]:
df = df[df['Symbol'] == name]

In [42]:
df.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
22392992,79631194629098,B,501.00,0,Limit,VOLTAS,250,1300000000003062,N,1
22393152,79631194645445,S,545.15,0,Limit,VOLTAS,50,1300000000003298,N,1
22393293,79631194675359,S,549.00,0,Limit,VOLTAS,50,1300000000003455,N,1
22393341,79631194682907,S,534.90,0,Limit,VOLTAS,5,1300000000003511,N,1
22393488,79631194698446,S,559.00,0,Limit,VOLTAS,5,1300000000003673,N,1


In [43]:
can = pd.read_parquet('cancelled.parquet')

In [44]:
can = can.rename(columns = {'TimeStamp' : 'Timestamp'})

In [45]:
can.head()

,Timestamp,Order Number,IOC Flag,Activity Type
0,79631194995693,1000000000007281,N,3
1,79631195013045,1000000000005540,N,3
2,79631195115036,1000000000006571,N,3
3,79631195116810,1000000000007282,N,3
4,79631195125870,1000000000007860,N,3


In [46]:
can_filtered = can[can['Order Number'].isin(df['Order Number'])]

In [47]:
can_filtered.head()


,Timestamp,Order Number,IOC Flag,Activity Type
2069165,79631205951608,1300000000043184,N,3
2069596,79631220322441,1300000000024912,N,3
2069735,79631222754806,1300000000005145,N,3
2069899,79631253536767,1300000000063547,N,3
2069990,79631253573392,1300000000065985,N,3


In [48]:
len(can_filtered)

39177

In [49]:
data = pd.concat([df, can_filtered], ignore_index=True)


In [50]:
print(len(data) == len(df) + len(can_filtered))

True


In [51]:
data.head(100000000)

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
0,79631194629098,B,501.00,0.0,Limit,VOLTAS,250.0,1300000000003062,N,1
1,79631194645445,S,545.15,0.0,Limit,VOLTAS,50.0,1300000000003298,N,1
2,79631194675359,S,549.00,0.0,Limit,VOLTAS,50.0,1300000000003455,N,1
3,79631194682907,S,534.90,0.0,Limit,VOLTAS,5.0,1300000000003511,N,1
4,79631194698446,S,559.00,0.0,Limit,VOLTAS,5.0,1300000000003673,N,1
...,...,...,...,...,...,...,...,...,...,...
470284,79632794924448,NaN,NaN,NaN,NaN,NaN,NaN,1300000005062502,N,3
470285,79632806660240,NaN,NaN,NaN,NaN,NaN,NaN,1300000005050982,N,3
470286,79632808166374,NaN,NaN,NaN,NaN,NaN,NaN,1300000000865427,N,3
470287,79632814064850,NaN,NaN,NaN,NaN,NaN,NaN,1300000005059092,N,3


In [52]:
data.to_parquet(f'{name}_order.parquet')